In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# df = pd.read_csv('../input/calculating-flow-rate-from-pressure-measurements/merged_tables.csv')
df = pd.read_csv('/merged_tables.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section

In [ ]:
df['pressure-in'] = df['pressure-in'].str.replace(',','.').astype(float)
df['pressure-out'] = df['pressure-out'].str.replace(',','.').astype(float)
df.dtypes

In [ ]:
null= df.isna().sum()
df.drop('time',axis=1,inplace=True)
per_null= df.isna().sum()/df.isna().count()*100
per_null= round(per_null, 1)
missing_data= pd.concat([null, per_null], axis=1,keys=['number of missing values', '% of missig values'])
missing_data

In [ ]:
df=df.interpolate()
df

In [ ]:
X = df.drop('flow',axis=1)
y = df['flow']

In [ ]:
df

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,train_test_split
grid = {
    "n_estimators":[100, 200, 500, 1000],
    "max_depth":[None, 5],
    "max_features":["auto"],
    "min_samples_split":[4, 6],
    "min_samples_leaf":[1, 2, 4]
}
np.random.seed(42)

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regressor = RandomForestRegressor()

#setup grid search cv
gs_regressor = GridSearchCV(
                            estimator=regressor,
                            param_grid=grid,
                            cv=5,
                            verbose=2
                            )

gs_regressor.fit(X_train,y_train)

In [ ]:
y_preds = gs_regressor.predict(X_test)

In [ ]:
y_preds

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_preds,y_test) # accuracy

In [ ]:
import pickle
with open('da_final_model', 'wb') as f:
    pickle.dump(gs_regressor, f) # saving the model to a file


In [ ]:
with open('da_final_model', 'rb') as f: # loading the model
    rf = pickle.load(f)

In [ ]:
!pip install scikit-plot # dependencies for plotting the training and eval graphs

In [ ]:
import  scikitplot as skplt
import matplotlib.pyplot as plt

In [ ]:
y_preds = gs_regressor.predict(X_test)

In [ ]:
y_preds.shape

In [ ]:
fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)

skplt.estimators.plot_feature_importances(RandomForestRegressor(),
                                         title="Random Forest Regressor Feature Importance",
                                         x_tick_rotation=90, order="ascending",
                                         ax=ax1);